# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 10 Assignment: Time Series Neural Network**

**Student Name: Wei Lu**

# Assignment Instructions

For this assignment you will use a LSTM to predict a time series contained in the data file **[series-31.csv](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/data/series-31.csv)**.  The code that you will use to complete this will be similar to the sunspots example from the course module.  This data set contains two columns: *time* and *value*.  Create a LSTM network and train it with a sequence size of 5 and a prediction window of 1.  If you use a different sequence size, you will not have the correct number of submission rows. Train the neural network, the data set is fairly simple and you should easily be able to get a RMSE below 1.0.  FYI, I generate this datasets by fitting a cubic spline to a series of random points. 

This is a time series data set, do not randomize the order of the rows!  For your training data use all *time* values less than 3000 and for test, use the remaining values greater than or equal to 3000. For the submit file, send me the results of your test evaluation.  You should have two columns: *time* and *value*.  The column *time* should be the time at the beginning of each predicted sequence. The *value* should be the next value that was predicted for each of your sequences.

Your submission file will look similar to:

|time|value|
|-|-|
|3000|37.022846|
|3001|37.030582|
|3002|37.03816|
|3003|37.045563|
|3004|37.0528|
|...|...|

# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems. 

**It is unlikely that should need to modify this function.**

In [0]:
import base64
import os
import numpy as np
import pandas as pd
import requests

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
!ls /content/drive/My\ Drive/Colab\ Notebooks

 assignment_WeiLu_class10.ipynb
 assignment_WeiLu_class1.ipynb
 assignment_WeiLu_class2.ipynb
 assignment_WeiLu_class3.ipynb
 assignment_WeiLu_class4.ipynb
 assignment_WeiLu_class5.ipynb
 assignment_WeiLu_class6.ipynb
 assignment_WeiLu_class7.ipynb
 assignment_WeiLu_class8.ipynb
 assignment_WeiLu_class9.ipynb
 “assignment_YaMingGao_class5.ipynb”的副本
 “assignment_YaMingGao_class6.ipynb”的副本
'“Copy of Copy of t81_558_class06_backpropagation.ipynb”的副本'
'“Copy of Copy of t81_558_class06_backpropagation.ipynb”的副本 (1)'
'“Copy of Copy of t81_558_class06_backpropagation.ipynb”的副本 (2)'
 “deriv.ipynb”的副本
 “t81_558_class_01_1_overview.ipynb”的副本
 “t81_558_class_01_4_python_files.ipynb”的副本
 “t81_558_class_01_5_python_functional.ipynb”的副本
 “t81_558_class_02_1_python_pandas.ipynb”的副本
 “t81_558_class_02_2_pandas_cat.ipynb”的副本
 “t81_558_class_02_3_pandas_grouping.ipynb”的副本
 “t81_558_class_03_2_keras.ipynb”的副本
 “t81_558_class_04_1_feature_encode.ipynb”的副本
 “t81_558_class_05_2_kfold.ipynb”的副本
 “t81_558_cla

# Assignment #10 Sample Code

The following code provides a starting point for this assignment.

In [3]:
import numpy as np
def to_sequences(seq_size, obs):
    x = []
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE):
        #print(i)
        window = obs[i:(i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)
        
    return np.array(x),np.array(y)
    

key = "nFpEY2LlkO1YtRFzGfAdnaL2S1FiaWjD7MPdWB8B"
file='/content/drive/My Drive/Colab Notebooks/assignment_WeiLu_class10.ipynb'

# Read from time series file
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/series-31-num.csv")


print("Starting file:")
print(df[0:10])

print("Ending file:")
print(df[-10:])

Starting file:
   time      value
0     0  10.000000
1     1  10.050953
2     2  10.101758
3     3  10.152415
4     4  10.202924
5     5  10.253286
6     6  10.303499
7     7  10.353566
8     8  10.403485
9     9  10.453256
Ending file:
      time      value
3990  3990  14.694572
3991  3991  14.727313
3992  3992  14.760351
3993  3993  14.793687
3994  3994  14.827322
3995  3995  14.861256
3996  3996  14.895491
3997  3997  14.930026
3998  3998  14.964862
3999  3999  15.000000


In [0]:
df_train = df[df['time']<3000]
df_test = df[df['time']>=3000]

ser_train = df_train['value'].tolist()
ser_test = df_test['value'].tolist()

# Create a LSTM network and train it with a sequence size of 5 and a prediction window of 1. 
SEQUENCE_SIZE = 5
x_train,y_train = to_sequences(SEQUENCE_SIZE,ser_train)
x_test,y_test = to_sequences(SEQUENCE_SIZE,ser_test)

In [8]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

print('Build model...')
model = Sequential()
model.add(LSTM(64, dropout=0.0, recurrent_dropout=0.0,input_shape=(None, 1)))
model.add(Dense(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)
print('Train...')

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Build model...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2995 samples, validate on 995 samples
Epoch 1/1000
2995/2995 - 2s - loss: 71.6592 - val_loss: 19.1588
Epoch 2/1000
2995/2995 - 1s - loss: 0.5461 - val_loss: 0.5675
Epoch 3/1000
2995/2995 - 1s - loss: 0.0266 - val_loss: 0.1716
Epoch 4/1000
2995/2995 - 1s - loss: 0.0102 - val_loss: 0.0817
Epoch 5/1000
2995/2995 - 1s - loss: 0.0050 - val_loss: 0.0531
Epoch 6/1000
2995/2995 - 1s - loss: 0.0043 - val_loss: 0.0404
Epoch 7/1000
2995/2995 - 1s - loss: 0.0033 - val_loss: 0.0312
Epoch 8/1000
2995/2995 - 1s - loss: 0.0023 - val_loss: 0.0347
Epoch 9/1000
2995/2995 - 1s - loss: 0.0020 - val_loss: 0.0441
Epoch 10/1000
2995/2995 - 1s - loss: 0.0030 - val_loss: 0.0191
Epoch 11/1000
2995/2995 - 1s - loss: 0.0017 - val_loss: 0.0285
Epoch 12/1000
2995/2995 - 1s - loss: 0.0018 - val_loss: 0.0162
Epoch

In [9]:
from sklearn import metrics

pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

Score (RMSE): 0.10879856935195371


In [0]:
time = pd.DataFrame(df_test['time'], columns=['time']).reset_index(drop=True)
value = pd.DataFrame(pred, columns=['value'])
df = pd.concat([time, value], axis=1).dropna(how='any')

In [32]:
df

,time,value
0,3000,36.499001
1,3001,36.505634
2,3002,36.512039
3,3003,36.518299
4,3004,36.524414
5,3005,36.530380
6,3006,36.536201
7,3007,36.541885
8,3008,36.547432
9,3009,36.552841


In [33]:
submit(source_file=file,data=df,key=key,no=10)

Success: Submitted Assignment #10 for luwei:
You have submitted this assignment 2 times. (this is fine)

